In [9]:
import argparse
import json
import os
import random
from gensim.summarization import bm25
import jieba
import numpy as np
from summary import stopwords, split_sentence, unique_sentences
from icecream import ic
import os
import os.path as osp
import numpy as np
import json
import torch
import re
from tqdm import tqdm
from icecream import ic
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("../models/chinese-roberta-wwm-ext")
crime_pattern = re.compile(r'已构成(.*?)罪')

In [10]:
example_query = "2018年1月15日14时10分许，被告人莫新国酒后驾驶湘A×××××号小型轿车沿长沙市天心区伊莱克斯大道由南往北行驶至水电八局基地路段时被在该处执勤的长沙市公安局交通警察支队民警检查，经现场酒精吹气检测，测试结果显示其血液中乙醇含量为195毫克／100毫升，随即被告人莫新国被交警带至湖南省融城医院抽取血样，并将血样送至长沙市公安局物证鉴定所检验，经检验，其血液中乙醇含量为201.1毫克／100毫升。2009年11月15日，被告人莫新国经长沙市残疾人联合会审核为精神残疾人。2018年5月28日，经湖南省芙蓉司法鉴定中心鉴定，被告人莫新国作案时处于普通醉酒状态，实施危害行为时有完全刑事责任能力。2018年1月30日，被告人莫新国主动到公安机关投案，其归案后如实供述了自己的罪行。"

example_candidate = " 夏邑县人民检察院指控： 2018年5月14日16时12分许，被告人王某某酒后驾驶豫N×××××白色小型轿车，沿夏邑县X019县道由西向东行驶至夏邑县车站镇程大庄村时，被夏邑县交警大队执勤民警当场查获。经现场对王某某进行呼气式酒精含量检测，检测结果为85mg／100ml。后抽血鉴定，王某某静脉血液中乙醇含量为102.97mg／100ml，属醉酒驾驶机动车。 针对上述指控，公诉机关当庭宣读并出示了：1、被告人常住人口基本信息、驾驶人及车辆信息查询；2、（2017）苏0583刑初1140号刑事判决书；3、查获经过；4、酒精呼吸测试单；5、强制措施凭证；6、当事人血样提取登记表；7、商丘普济法医临床司法鉴定所血样检验报告；8、被告人王某某供述等证据。 公诉机关认为，被告人王某某作为具有完全刑事责任能力的成年人，明知饮酒后不允许驾驶机动车辆，仍然违反交通运输管理法规，醉酒状态下驾驶机动车在道路上行驶，其行为触犯了《中华人民共和国刑法》第一百三十三条之一的规定，犯罪事实清楚，证据确实充分，应当以危险驾驶罪追究其刑事责任。被告人在缓刑考验期内又犯新罪，应当撤销缓刑，对新犯的罪作出判决，数罪并罚。提起公诉，请依法判处。 被告人王某某对公诉机关指控的犯罪事实及罪名无异议。 辩护人认为，被告人血液酒精含量较低，没有发生危害后果，虽属醉酒驾驶机动车，但情节显著轻微，应不予定罪处罚。 经审理查明的犯罪事实与公诉机关指控一致，且有下列证据予以证实： 1、被告人常住人口基本信息、驾驶人及机动车信息查询，证明被告人王某某的出生日期及家庭住址，系完全刑事责任年龄人，准驾车型C1，系车主。 2、（2017）苏0583刑初1140号刑事判决书，证明被告人王某某因犯非法拘禁罪于2017年8月9日被江苏省昆山市人民法院判处有期徒刑六个月，缓刑一年。 3、查获经过，证明2018年5月14日16时12分许，王某某酒后驾驶豫N×××××号小型轿车，沿夏邑县X019县道由西向东行驶至夏邑县车站镇程大庄村时，被夏邑县交警大队执勤民警当场查获。经现场对王某某进行呼气式酒精含量检测，检测结果为85mg／100ml。随后把王某某带至夏邑县交警大队。 4、强制措施凭证，证明案发后强制检验王某某血液、尿样。 5、酒精呼吸结果单，证明现场对王某某进行酒精测试，酒精含量为85.0mg／100ml。 6、血样提取登记表，证明2018年5月14日16时40分，交警大队民警带被告人到夏邑县红十字医院提取血样的情况。 7、商丘普济法医临床司法鉴定所血样检验报告，证明经抽取王某某静脉血液检验，血液中乙醇含量为102.97mg／100ml。 8、被告人王某某的供述，证明2018年5月14日16时许，其饮酒后驾驶机动车被民警查获，现场使用呼气式酒精检测仪进行检测，结果为85.0mg／100ml。 上述证据，经庭审举证、质证，证据来源清楚，收集程序合法，能够证明案件事实，本院予以确认。 "


In [19]:
def cut_words(sentence):
    words = jieba.cut(sentence, cut_all=False)
    tem = " ".join(words).split()
    return [i for i in tem if not i in stopwords]


def rerank_sentences(query_words, candidate):
    candidate_sentences = list(split_sentence(candidate))
    corpus = [cut_words(s) for s in candidate_sentences]
    bm25Model = bm25.BM25(corpus)
    sentence_scores = np.array(bm25Model.get_scores(query_words))
    filtered_candidate_sentences = [sent for sent, score in zip(
        candidate_sentences, sentence_scores) if score > 0]
    filtered_sentence_scores = sentence_scores[sentence_scores > 0]
    reranked_sentences = [filtered_candidate_sentences[i]
                          for i in filtered_sentence_scores.argsort().tolist()[::-1]]
    uniqued_reranked_sentences = list(unique_sentences(reranked_sentences))
    return ''.join(uniqued_reranked_sentences)


rerank_sentences(cut_words(example_query), example_candidate)


'2018年5月14日16时12分许，被告人王某某酒后驾驶豫N×××××白色小型轿车，沿夏邑县X019县道由西向东行驶至夏邑县车站镇程大庄村时，被夏邑县交警大队执勤民警当场查获。查获经过，证明2018年5月14日16时12分许，王某某酒后驾驶豫N×××××号小型轿车，沿夏邑县X019县道由西向东行驶至夏邑县车站镇程大庄村时，被夏邑县交警大队执勤民警当场查获。血样提取登记表，证明2018年5月14日16时40分，交警大队民警带被告人到夏邑县红十字医院提取血样的情况。被告人王某某的供述，证明2018年5月14日16时许，其饮酒后驾驶机动车被民警查获，现场使用呼气式酒精检测仪进行检测，结果为85.0mg／100ml。商丘普济法医临床司法鉴定所血样检验报告，证明经抽取王某某静脉血液检验，血液中乙醇含量为102.97mg／100ml。后抽血鉴定，王某某静脉血液中乙醇含量为102.97mg／100ml，属醉酒驾驶机动车。（2017）苏0583刑初1140号刑事判决书，证明被告人王某某因犯非法拘禁罪于2017年8月9日被江苏省昆山市人民法院判处有期徒刑六个月，缓刑一年。辩护人认为，被告人血液酒精含量较低，没有发生危害后果，虽属醉酒驾驶机动车，但情节显著轻微，应不予定罪处罚。经现场对王某某进行呼气式酒精含量检测，检测结果为85mg／100ml。商丘普济法医临床司法鉴定所血样检验报告；酒精呼吸结果单，证明现场对王某某进行酒精测试，酒精含量为85.0mg／100ml。公诉机关认为，被告人王某某作为具有完全刑事责任能力的成年人，明知饮酒后不允许驾驶机动车辆，仍然违反交通运输管理法规，醉酒状态下驾驶机动车在道路上行驶，其行为触犯了《中华人民共和国刑法》第一百三十三条之一的规定，犯罪事实清楚，证据确实充分，应当以危险驾驶罪追究其刑事责任。被告人王某某供述等证据。强制措施凭证，证明案发后强制检验王某某血液、驾驶人及机动车信息查询，证明被告人王某某的出生日期及家庭住址，系完全刑事责任年龄人，准驾车型C1，系车主。被告人常住人口基本信息、酒精呼吸测试单；当事人血样提取登记表；被告人王某某对公诉机关指控的犯罪事实及罪名无异议。被告人在缓刑考验期内又犯新罪，应当撤销缓刑，对新犯的罪作出判决，数罪并罚。随后把王某某带至夏邑县交警大队。驾驶人及车辆信息查询；'

In [20]:
def preprocess_data(data_path, has_label=False):
    query_path = osp.join(data_path, 'query.json')
    all_candidates_path = osp.join(data_path, 'candidates')
    if has_label:
        label_path = osp.join(data_path, 'label_top30_dict.json')
        label = json.load(open(label_path))
        labels = []
    edges, inputs, query_ridxs, node_graph_ids, edge_graph_ids, candidate_ridxs = [
    ], [], [], [], [], []
    with open(query_path) as f:
        query_lines = f.readlines()
    for query_line in tqdm(query_lines):
        query_line = query_line.strip()
        query_dict = json.loads(query_line)
        input_str = "[SEP]".join(query_dict['crime']) + \
            "[SEP]"+query_dict['q']
        query_words = cut_words(query_dict['q'])
        tokenized_inputs = tokenizer(input_str, return_tensors="pt")
        query_idx = len(inputs)
        inputs.append(tokenized_inputs)
        node_graph_ids.append(len(query_ridxs))
        query_ridxs.append(query_dict['ridx'])
        query_ridx = str(query_dict['ridx'])
        candidates_path = osp.join(all_candidates_path, query_ridx)
        for candidate in os.listdir(candidates_path):
            candidate_ridx = candidate[:-5]
            candidate_path = osp.join(candidates_path, candidate)
            candidate_dict = json.load(open(candidate_path))
            all_text = ''.join(candidate_dict.values())
            crime_name = crime_pattern.search(all_text)
            if crime_name is None:
                crime_name = ''
            else:
                crime_name = crime_name.group(1) + '罪'
            candidate_text = rerank_sentences(
                query_words, candidate_dict['ajjbqk'])
            candidate_text = '[SEP]'.join(
                [crime_name, candidate_text])
            tokenized_candidate = tokenizer(
                candidate_text, return_tensors="pt")
            candidate_idx = len(inputs)
            inputs.append(tokenized_candidate)
            node_graph_ids.append(node_graph_ids[-1])
            edge_graph_ids.append(node_graph_ids[-1])
            edges.append([query_idx, candidate_idx])
            candidate_ridxs.append(int(candidate_ridx))
            if has_label:
                if candidate_ridx in label[query_ridx]:
                    labels.append(label[query_ridx][candidate_ridx])
                else:
                    labels.append(0)
    if has_label:
        return inputs, edges, query_ridxs, node_graph_ids, edge_graph_ids, candidate_ridxs, labels
    return inputs, edges, query_ridxs, node_graph_ids, edge_graph_ids, candidate_ridxs


train_path = '../data/origin/train'
train_inputs, train_edges, train_query_ridxs, train_node_graph_ids, train_edge_graph_ids, train_candidate_ridxs, train_labels = preprocess_data(
    train_path, has_label=True)


 48%|████▊     | 95/197 [05:20<04:55,  2.89s/it]

In [ ]:
def save(processed_path, edges, inputs, query_ridxs, node_graph_ids, edge_graph_ids, candidate_ridxs, labels=None):
    torch.save(inputs, osp.join(processed_path, 'inputs.pt'))
    torch.save(edges, osp.join(processed_path, 'edges.pt'))
    torch.save(query_ridxs, osp.join(processed_path, 'query_ridxs.pt'))
    torch.save(candidate_ridxs, osp.join(processed_path, 'candidate_ridxs.pt'))
    torch.save(node_graph_ids, osp.join(processed_path, 'node_graph_ids.pt'))
    torch.save(edge_graph_ids, osp.join(processed_path, 'edge_graph_ids.pt'))
    if labels is not None:
        torch.save(labels, osp.join(processed_path, 'labels.pt'))

In [ ]:
train_processed_path = '../data/bm25/train/processed'
os.makedirs(train_processed_path, exist_ok=True)
save(train_processed_path, train_edges, train_inputs, train_query_ridxs,
     train_node_graph_ids, train_edge_graph_ids, train_candidate_ridxs, train_labels)

In [ ]:
test_path = '../data/origin/test'
test_inputs, test_edges, test_query_ridxs, test_node_graph_ids, test_edge_graph_ids, test_candidate_ridxs = preprocess_data(
    test_path, has_label=False)
test_processed_path = '../data/bm25/test/processed'
os.makedirs(test_processed_path, exist_ok=True)
save(test_processed_path, test_edges, test_inputs, test_query_ridxs,
     test_node_graph_ids, test_edge_graph_ids, test_candidate_ridxs)